<a href="https://colab.research.google.com/github/DanielCalero1/Smooking-recognition/blob/ResNet/Experiments_ResNet.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install torchvision


In [11]:
from torchvision import models

import torch
import torch.nn as nn
import torch.nn.functional as F
from torchvision import datasets, transforms
from torch.utils.data import DataLoader,TensorDataset
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score

from google.colab import drive
drive.mount('/content/drive')
import zipfile
import os

# for number-crunching
import numpy as np
import scipy.stats as stats

# for dataset management
import pandas as pd

# for data visualization
import matplotlib.pyplot as plt
from IPython import display
display.set_matplotlib_formats('svg')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


<ipython-input-11-d4fff5e1aa1e>:26: DeprecationWarning: `set_matplotlib_formats` is deprecated since IPython 7.23, directly use `matplotlib_inline.backend_inline.set_matplotlib_formats()`
  display.set_matplotlib_formats('svg')


In [2]:
resnet50 = models.resnet50(pretrained=True)


/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/usr/local/lib/python3.10/dist-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet50_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet50_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)
Downloading: "https://download.pytorch.org/models/resnet50-0676ba61.pth" to /root/.cache/torch/hub/checkpoints/resnet50-0676ba61.pth
100%|██████████| 97.8M/97.8M [00:00<00:00, 137MB/s]


In [7]:
transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
])
train_dir = '/content/drive/MyDrive/Dataset-1/training_data'
val_dir = '/content/drive/MyDrive/Dataset-1/validation_data'

train_dataset = datasets.ImageFolder(root=train_dir, transform=transform)
val_dataset = datasets.ImageFolder(root=val_dir, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True)
val_loader = DataLoader(val_dataset, batch_size=32, shuffle=False)

##Testing the model without additional training

In [18]:
'''
First, we need to change the last layer of resnet to have only two (or only one?) output. Resnet was trained
to predict images within a set of 100 classes. In our case we only have two classes.
'''
resnet50.fc = nn.Linear(resnet50.fc.in_features, 1)

resnet50.eval()

predictions = []
labels = []
with torch.no_grad():
    for images, targets in val_loader:
        outputs=resnet50(images)

        probs = torch.sigmoid(outputs).squeeze()
        predicted_classes = (probs > 0.5).long()
        predictions.extend(predicted_classes.numpy())
        labels.extend(targets.numpy())

accuracy = accuracy_score(labels, predictions)
print(f'Accuracy: {accuracy * 100:.2f}%')

Accuracy: 57.75%
